<a href="https://colab.research.google.com/github/chrober24/SL1_2021/blob/main/stylegan2_tutorial_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## stylegan2: Training

this notebook contains scripts for training a stylegan2 model as well as various scripts for generating images from a trained network and projecting into the network

Training requires connecting colab to google drive as the training process generates a large amount of data that cannot be hosted locally on colab.

# setup

In [ ]:
#always use tensorflow1

%tensorflow_version 1.x


TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Initial Setup

Run this cell if you’ve never run this repo in your Drive account. SKIP it if you have.

This cell will download the Stylegan2 repository to your Drive and create the necessary folder structure to train a custom stylegan2 model

In [ ]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/MyDrive/
!mkdir stylegan2-aug-colab
%cd stylegan2-aug-colab/
!git clone -b augs-attn https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
%cd ../../
%mkdir datasets

/content/drive/MyDrive
/content/drive/MyDrive/stylegan2-aug-colab
Cloning into 'stylegan2'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 22.47 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (285/285), done.
/content/drive/MyDrive/stylegan2-aug-colab/stylegan2
/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/drive/My Drive/stylegan2-aug-colab/stylegan2/pkl/inception_v3_features.pkl
87.3MB [00:00, 95.3MB/s]
/content/drive/My Drive/stylegan2-aug-colab/stylegan2
/content/drive/My Drive/stylegan2-aug-colab/stylegan2/results/00001-pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/drive/My Drive/stylegan2-aug-colab/stylegan2/results/00001-pretrained/stylegan2-ffhq-config-f.pkl
382MB [00:03, 99.4MB/s]
/content/drive/My Drive/stylegan

## Return Setup
Run this cell if you’re picking up from a previous training.

In [ ]:
%cd /content/drive/MyDrive/stylegan2-aug-colab/stylegan2
#!git pull

/content/drive/MyDrive/stylegan2-aug-colab/stylegan2


# dataset prep:
creates tfrecord (only run at beginning of training)


In [ ]:
!python dataset_tool.py create_from_images_raw /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/human_insect_limbs /content/drive/MyDrive/dataset_prep/dataset-tools/output/human_insect_limbs_png/sq-512/

Loading images from "/content/drive/MyDrive/dataset_prep/dataset-tools/output/human_insect_limbs_png/sq-512/"
detected 922 images ...
Shuffle the images...
Creating dataset "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/human_insect_limbs"
Adding the images to tfrecords ...
added images 0
Added 208 images.
Traceback (most recent call last):
  File "dataset_tool.py", line 943, in <module>
    execute_cmdline(sys.argv)
  File "dataset_tool.py", line 937, in execute_cmdline
    func(**vars(args))
  File "dataset_tool.py", line 712, in create_from_images_raw
    encoded_jpg = fid.read()
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/lib/io/file_io.py", line 128, in read
    pywrap_tensorflow.ReadFromStream(self._read_buf, length))
KeyboardInterrupt


# Pre-train model download
run only once!

In [ ]:
!gdown --id 1Muz1D56wm_x2ysLyVqTuaMIazsQgmIOg -O /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/pkl/stylegan2-car-config-f.pkl

# Training

In the next cell set your pkl and `resume_kimg` counter. If this is your first time the settings below should work for you.

This cell should be used for transfer learning as well as resuming a current train. 

In [ ]:
pkl="/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00027-stylegan2-human_insect_limbs-1gpu-config-f/network-snapshot-010900.pkl"
resume_kimg = 10900

the `AUG_PROB` environment setting tells the training loop how often to modify the real and fake images with augmentations. The default is `0.1` If you have a small training set you may want to go higher than that but note that the Karras paper does say if you set this value too high you may find it bleeds into the outputs.

Replace --dataset with your dataset name

In [ ]:
!AUG_PROB=0.2 python run_training.py --num-gpus=1 --mirror-augment=True --data-dir=./datasets --dataset=human_insect_limbs --config=config-f --resume-pkl=$pkl --resume-kimg=$resume_kimg --augmentations=True --metrics=None

Local submit - run_dir: results/00028-stylegan2-human_insect_limbs-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
Dataset shape = [3, 512, 512]
Dynamic range = [0, 255]
Label size    = 0
Loading networks from "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/results/00027-stylegan2-human_insect_limbs-1gpu-config-f/network-snapshot-010900.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.

G                             Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
lod                           -         ()                  -               
dla